# map information 
### The city I choose is Ann Arbor, MI,US . The map range is (-83.8724,42.2166,-83.6029,42.3301). The downloaded file is called 'map'

# data audit
use the iterative parsing to process the map file and
find out not only what tags are there, but also how many, to get the
feeling on how much of which data you can expect to have in the map.


In [20]:
import os
file_path = 'map'
b = os.path.getsize(file_path)
print 'file size is {} MB'.format(b/(1024*1024))

file size is 85 MB


In [21]:
import xml.etree.cElementTree as ET
import pprint

def count_tags(filename):
       
    tags = {}
    for event,element in ET.iterparse(filename):
        tag = element.tag
        if tag not in tags.keys():
            tags[tag] = 1
        else:
            tags[tag]+=1
    return tags
tags = count_tags('map')
print 'tag collection result:'
pprint.pprint(tags)

tag collection result:
{'bounds': 1,
 'member': 9460,
 'meta': 1,
 'nd': 473164,
 'node': 388849,
 'note': 1,
 'osm': 1,
 'relation': 657,
 'remark': 1,
 'tag': 208224,
 'way': 59064}



Before you process the data and add it into your database, you should check the
"k" value for each "tag" and see if there are any potential problems.

 we have a count of each of four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.


In [22]:
import re
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')



def key_type(element, keys):
    if element.tag == "tag":
        
        if lower.match(element.attrib['k']):
            keys["lower"] = keys["lower"] + 1
        elif lower_colon.match(element.attrib['k']):
            keys["lower_colon"] = keys["lower_colon"] + 1
        elif problemchars.match(element.attrib['k']):
            keys["problemchars"] = keys["problemchars"] + 1
        else:
            keys["other"] = keys["other"] + 1
    return keys
def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys
keys = process_map('map')
pprint.pprint(keys)

{'lower': 141733, 'lower_colon': 61757, 'other': 4734, 'problemchars': 0}


### find out how many unique users have contributed to the map in Ann Arbor!

In [23]:
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.get('uid'):
            users.add(element.get('uid'))

    return users
users = process_map('map')
print len(users)

435


# audit and update streetname
We have offered normal street names in expected. the following work is mapping the problematic street name to the normal name. 

In [24]:
from collections import defaultdict

OSMFILE = "map"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons",'Circle']

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd": "Road",
           "road":"Road",
           'Plaza':"Square",
           'Way':'Road',
           'Ct':"Court"
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):

    betterNameParts= []

    name = name.split()
    for index, part in enumerate(name):
        if part in mapping.keys():
            betterNameParts.append(mapping[part])
        else:
            betterNameParts.append(part)
       
    betterName = ' '.join(betterNameParts)

    return betterName




st_types = audit(OSMFILE)

pprint.pprint(dict(st_types))

for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name


{'Arcade': set(['Nickels Arcade']),
 'Ave': set(['Jackson Ave']),
 'Boardwalk': set(['Boardwalk']),
 'Ct': set(['Armstrong Ct']),
 'East': set(['Woodland Drive East']),
 'Edenwood': set(['Edenwood']),
 'Eisenhower': set(['West Eisenhower']),
 'Highway': set(['South Industrial Highway']),
 'North': set(['Village Circle North']),
 'Plaza': set(['Jackson Plaza', 'Parkland Plaza']),
 'Rd': set(['Jackson Rd']),
 'South': set(['Village Circle South']),
 'Way': set(['Carrot Way', 'Ember Way', 'Harbor Way', 'Victors Way']),
 'road': set(['Carpenter road', 'Packard road'])}
Village Circle North => Village Circle North
Nickels Arcade => Nickels Arcade
Village Circle South => Village Circle South
Parkland Plaza => Parkland Square
Jackson Plaza => Jackson Square
Edenwood => Edenwood
Jackson Rd => Jackson Road
Boardwalk => Boardwalk
South Industrial Highway => South Industrial Highway
Ember Way => Ember Road
Carrot Way => Carrot Road
Victors Way => Victors Road
Harbor Way => Harbor Road
Jackson Ave

# Preparing for Database
## transfer xml data into csv + data cleaning

In [25]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET



OSM_PATH = "map"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')




# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # the following code I have cited some from https://github.com/sfox1975/Udacity-DAND-Project-3/blob/master/data.py
    if element.tag == 'node':

            for node_field in node_attr_fields:
                node_attribs[node_field] =element.attrib[node_field]

            for tag in element.iter('tag'):
                k = tag.attrib['k']
                

                # ignores tags containing problem characters in the k tag attribute:

                if re.search(PROBLEMCHARS,k):
                    continue
                else:
                    pass

                tag_dict = {}

                tag_dict['id'] = node_attribs['id']

                colon_find = re.split('[:]', k)

                if len(colon_find) == 1:

                    tag_dict['key'] = k
                    tag_dict['type'] = 'regular'
                    tag_dict['value'] = tag.attrib['v']

                elif len(colon_find) == 2:

                    tag_dict['key'] = colon_find[1]
                    tag_dict['type'] = colon_find[0]
                    # when the tag is street, we will update the street name 
                    if colon_find[1] == 'street':
                        tag_dict['value'] = update_name(tag.attrib['v'], mapping)

                elif len(colon_find) > 2:

                    tag_dict['key'] = ':'.join(colon_find[1:])
                    tag_dict['type'] = colon_find[0]
                    tag_dict['value'] = tag.attrib['v']

                

                tags.append(tag_dict)

            return {'node': node_attribs, 'node_tags': tags}

    elif element.tag == 'way':

        for way_field in way_attr_fields:
            way_attribs[way_field] =element.attrib[way_field]

        for tag in element.iter('tag'):
            k = tag.attrib['k']

            # ignores tags containing problem characters in the k tag attribute:

            if re.search(PROBLEMCHARS,k):
                print "Problem character found - skipping element"
                continue
            else:
                pass

            tag_dict = {}

            tag_dict['id'] = way_attribs['id']

            colon_find = re.split('[:]', k)

            if len(colon_find) == 1:

                tag_dict['key'] = k
                tag_dict['type'] = 'regular'
                tag_dict['value'] = tag.attrib['v']

            elif len(colon_find) == 2:

                tag_dict['key'] = colon_find[1]
                tag_dict['type'] = colon_find[0]
                # when the tag is street, we will update the street name 
                if colon_find[1] == 'street':
                    tag_dict['value'] = update_name(tag.attrib['v'], mapping)

            elif len(colon_find) > 2:

                tag_dict['key'] = ':'.join(colon_find[1:])
                tag_dict['type'] = colon_find[0]
                tag_dict['value'] = tag.attrib['v']

            

            tags.append(tag_dict)

        n = 0
        for nd in element.iter('nd'):

            nd_dict = {}

            nd_dict['id'] = way_attribs['id']
            nd_dict['node_id'] = nd.attrib['ref']
            nd_dict['position'] = n
            way_nodes.append(nd_dict)
            n+=1

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()





class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

## use odo to transfer csv into sqlite3 database

In [5]:
import sqlite3
import csv
from odo import odo, resource, discover
#  Specify file path
#file_path = 'my_path/'
# In this case 'my_path/' is a substitute for my real path

# Specify csv file path and name
#csv_path = file_path + 'ways.csv'
csv_path1 = 'ways.csv'
csv_path2 = 'ways_nodes.csv'
csv_path3 = 'ways_tags.csv'
csv_path4 = 'nodes.csv'
csv_path5 = 'nodes_tags.csv'
# Specify database name
db_name = 'map.db'

# Connect to new database
conn = sqlite3.connect(db_name)

# [2]

# Use Odo to detect the shape and datatype of your csv:
data_shape1 = discover(resource(csv_path1))
data_shape2 = discover(resource(csv_path2))
data_shape3 = discover(resource(csv_path3))
data_shape4 = discover(resource(csv_path4))
data_shape5 = discover(resource(csv_path5))
# Ready in csv to new table called 'data' within database 'data.sqlite'
odo(csv_path1, 'sqlite:///map.db::ways', dshape=data_shape1)
odo(csv_path2, 'sqlite:///map.db::ways_nodes', dshape=data_shape2)
odo(csv_path3, 'sqlite:///map.db::ways_tags', dshape=data_shape3)
odo(csv_path4, 'sqlite:///map.db::nodes', dshape=data_shape4)
odo(csv_path5, 'sqlite:///map.db::nodes_tags', dshape=data_shape5)
# Close database
conn.close()

C:\Users\wenhao\Anaconda2\lib\site-packages\odo\backends\pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


# queries for created database
###  check the tables' column 

In [10]:
# Fetch records from either chinook.db
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'PRAGMA table_info(ways);'
c.execute(QUERY)
rows = c.fetchall()
print "column name for ways tables:"
for row in rows :
    print row
QUERY = 'PRAGMA table_info(ways_nodes);'
c.execute(QUERY)
rows = c.fetchall()
print "column name for ways_nodes tables:"
for row in rows :
    print row
QUERY = 'PRAGMA table_info(ways_tags);'
c.execute(QUERY)
rows = c.fetchall()
print "column name for ways_tags tables:"
for row in rows :
    print row
QUERY = 'PRAGMA table_info(nodes);'
c.execute(QUERY)
rows = c.fetchall()
print "column name for nodes tables:"
for row in rows :
    print row
QUERY = 'PRAGMA table_info(nodes_tags);'
c.execute(QUERY)
rows = c.fetchall()

print "column name for nodes_tags tables:"
for row in rows :
    print row
    


db.close()

column name for ways tables:
(0, u'id', u'BIGINT', 1, None, 0)
(1, u'user', u'TEXT', 0, None, 0)
(2, u'uid', u'BIGINT', 1, None, 0)
(3, u'version', u'BIGINT', 1, None, 0)
(4, u'changeset', u'BIGINT', 1, None, 0)
(5, u'timestamp', u'DATETIME', 0, None, 0)
column name for ways_nodes tables:
(0, u'id', u'BIGINT', 1, None, 0)
(1, u'node_id', u'BIGINT', 1, None, 0)
(2, u'position', u'BIGINT', 1, None, 0)
column name for ways_tags tables:
(0, u'id', u'BIGINT', 1, None, 0)
(1, u'key', u'TEXT', 0, None, 0)
(2, u'value', u'TEXT', 0, None, 0)
(3, u'type', u'TEXT', 0, None, 0)
column name for nodes tables:
(0, u'id', u'BIGINT', 1, None, 0)
(1, u'lat', u'FLOAT', 1, None, 0)
(2, u'lon', u'FLOAT', 1, None, 0)
(3, u'user', u'TEXT', 0, None, 0)
(4, u'uid', u'BIGINT', 1, None, 0)
(5, u'version', u'BIGINT', 1, None, 0)
(6, u'changeset', u'BIGINT', 1, None, 0)
(7, u'timestamp', u'DATETIME', 0, None, 0)
column name for nodes_tags tables:
(0, u'id', u'BIGINT', 1, None, 0)
(1, u'key', u'TEXT', 0, None, 0)
(

###  check  the number of unique contributer

In [13]:
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'select count(*) from (select distinct uid from ways union select distinct uid from nodes ) tmp'
c.execute(QUERY)
rows = c.fetchall()
print "total number of unique contributors is:"
for row in rows :
    print row[0]
db.close()

total number of unique contributors is:
425


### check the number of ways and nodes

In [3]:
import sqlite3
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'select count(distinct id) from ways'
c.execute(QUERY)
rows = c.fetchall()
print "total number of ways is:"
for row in rows :
    print row[0]
QUERY = 'select count(distinct id) from nodes'
c.execute(QUERY)
rows = c.fetchall()
print "total number of nodes is:"
for row in rows :
    print row[0]
db.close()

total number of ways is:
59064
total number of nodes is:
388849


### check the top ten most frequent type for nodes in Ann Arbor

In [18]:
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'select key, count(*) from nodes, nodes_tags where nodes.id = nodes_tags.id group by key order by count(*) desc limit 10;'
c.execute(QUERY)
rows = c.fetchall()
print "top ten most frequent type for nodes in Ann Arbor is:"
for row in rows :
    print row[0],row[1]
db.close()

top ten most frequent type for nodes in Ann Arbor is:
power 3516
highway 3423
name 1382
amenity 940
housenumber 932
street 925
created_by 886
entrance 456
shop 413
barrier 356


###  check the file size 

In [29]:
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'pragma page_size;'
c.execute(QUERY)

rows = c.fetchall()
print "page_size information:"
for row in rows :
    page_size =  row[0]
    print row[0]
QUERY = 'pragma page_count;'
c.execute(QUERY)

rows = c.fetchall()
print "page_count information:"
for row in rows :
    page_count =  row[0]
    print row[0]
print 'total database size is {} MB'.format(page_size*page_count/(1024*1024))
db.close()

page_size information:
1024
page_count information:
52268
total database size is 51 MB


## Other ideas about the dataset

top 3 contributor for nodes

In [4]:
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'select user, count(*) from nodes group by user order by count(*) desc limit 3;'
c.execute(QUERY)
rows = c.fetchall()
print "top 3 contributor for nodes in Ann Arbor is:"
for row in rows :
    print row[0],row[1]
db.close()

top 3 contributor for nodes in Ann Arbor is:
mapper377 109416
freebeer 29246
IanH 28910


In [ ]:
top 10 frequent tag values

In [11]:
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'select value, count(*) from nodes_tags where key = "amenity" group by value order by count(*) desc limit 10;'
c.execute(QUERY)
rows = c.fetchall()
print "top 30 frequent tag values for amenity in Ann Arbor is:"
for row in rows :
    print row[0],row[1]
db.close()

top 30 frequent tag values for amenity in Ann Arbor is:
restaurant 172
bench 169
cafe 65
fast_food 55
post_box 42
fountain 40
parking 33
bicycle_parking 30
bbq 23
fuel 23


In [14]:
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'select value, count(*) from nodes_tags where key = "shop" group by value order by count(*) desc limit 10;'
c.execute(QUERY)
rows = c.fetchall()
print "top 10 frequent tag values for shop in Ann Arbor is:"
for row in rows :
    print row[0],row[1]
db.close()

top 10 frequent tag values for shop in Ann Arbor is:
clothes 59
hairdresser 35
convenience 27
yes 17
jewelry 14
supermarket 13
vacant 13
beauty 12
mobile_phone 12
sports 10


### find the value when tags' key = name

In [17]:
db = sqlite3.connect("map.db")
c = db.cursor()
QUERY = 'select value, count(*) from nodes_tags where key = "name" group by value order by count(*) desc limit 10;'
c.execute(QUERY)
rows = c.fetchall()
print "top 10 frequent values for name in Ann Arbor is:"
for row in rows :
    print row[0],row[1]
db.close()

top 10 frequent values for name in Ann Arbor is:
#24 24
Subway 9
#7 8
Starbucks 8
#5 6
Mailbox 6
#66 5
Espresso Royale 5
Jimmy John's 5
Sears 5


### improvement adivce for the dataset
here we find some problem , when the tags' key equals name, we find the name includes Subway, starbucks which belongs to the Key amenity; #24,#7 #5 belongs to the Key highway. Thus to better represent the data set, the tags whose key equals name should be classified to the general classes like amenity and highway

### problems after the reclassification for the name tags :
After switch these name to general one, like switch name tags Subway to amenity tags fastfood, there's no room to show this fastfood's name is Subway. Thus may cause information loss.